# Experiment with getting sentinel-1 data from Earthdata Search (CMR API)


Details on forming your own URL search query:
https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html#granule-search-by-parameters

Several Python APIs already exists for contructing queries and managing results:
https://github.com/jddeal/python-cmr 

Also a NASA variant:
https://github.com/nasa/pyCMR 

# python-cmr

In [1]:
from cmr import CollectionQuery, GranuleQuery

In [2]:
api = CollectionQuery()
daacs = ['ASF']

for daac in daacs:
    collections = api.archive_center(daac).keyword('*').get(2000)
    print(daac, 'returns', len(collections), 'collections')
    
# Collections
[print(x['summary']) for x in collections]

ASF returns 3 collections
ALOS PALSAR Level 1.1
ALOS PALSAR Level 1.5
ALOS PALSAR Level 1.0


[None, None, None]

In [3]:
# Test with example from documentation
api = GranuleQuery()
granules = api.short_name("AST_L1T").point(-112.73, 42.5).get(3)
for granule in granules:
    print(granule["title"])

SC:AST_L1T.003:2149105822
SC:AST_L1T.003:2149105820
SC:AST_L1T.003:2149155037


In [4]:
# NOTE: not all scenes sorted into 'collections'. Sentinel-1 available via granule search
# Granule query for sentinel-1 data
api = GranuleQuery()
#api.archive_center("ASF") #only collection search
api.bounding_box(-122.4, 47.4, -122.1, 47.7) #seattle
api.temporal("2017-10-10T00:00:00Z", "2018-01-07T00:00:00Z")
#api.platform("SENTINEL-1A")
#api.instrument("SENTINEL-1 C-SAR")
#api.short_name("SENTINEL*") #product short name
api.entry_title('SENTINEL-1B_SLC')

# Summary of Sentinel SLC data set: (note itself a CMR query)
#https://cmr.earthdata.nasa.gov/search/concepts/C1214470488-ASF.html?token=568bf7f18164b928e5afcdbf567cdb67d5bd29e52271d0e9fe0da7eb6b2c1add:ZizV3PCdydlPQLBdbvADkA
#api.parameters(
#    short_name="SENTINEL*",
#)

print(api.hits())

35


In [5]:
granule = api.get(1)
print(len(granule))
test = granule[0]
test
# Note different return compared to pyCMR section below

1


{'browse_flag': False,
 'coordinate_system': 'GEODETIC',
 'data_center': 'ASF',
 'dataset_id': 'SENTINEL-1B_SLC',
 'day_night_flag': 'UNSPECIFIED',
 'granule_size': '4480.759',
 'id': 'G1427183289-ASF',
 'links': [{'href': 'https://datapool.asf.alaska.edu/SLC/SB/S1B_IW_SLC__1SDV_20171012T015310_20171012T015338_007790_00DC24_4ABE.zip',
   'hreflang': 'en-US',
   'rel': 'http://esipfed.org/ns/fedsearch/1.1/data#'},
  {'href': 'https://vertex.daac.asf.alaska.edu/',
   'hreflang': 'en-US',
   'inherited': True,
   'rel': 'http://esipfed.org/ns/fedsearch/1.1/data#'}],
 'online_access_flag': True,
 'orbit_calculated_spatial_domains': [{'orbit_number': '7790'}],
 'original_format': 'ECHO10',
 'polygons': [['46.882545 -122.173874 47.280518 -118.833061 48.954056 -119.217827 48.554779 -122.671318 46.882545 -122.173874']],
 'producer_granule_id': 'S1B_IW_SLC__1SDV_20171012T015310_20171012T015338_007790_00DC24_4ABE',
 'time_end': '2017-10-12T01:53:38.000Z',
 'time_start': '2017-10-12T01:53:10.000Z

In [6]:
# Hmmm.. not working. Here is the URL from Earthdata graphical search "CMR Endpoint"
#https://cmr.earthdata.nasa.gov:443/search/granules.json?
#        bounding_box=-122.4%2C47.4%2C-122.1%2C47.7
#        &echo_collection_id=C1214470488-ASF
#        &sort_key%5B%5D=-start_date
#        &page_size=20"

# I suspect 'echo_collection_id' is for metadata that isn't yet in CMR ideal format?

In [7]:
#api.echo_collection_id('C1214470488-ASF')
# Error

In [8]:
# Or for a single SLC on Earthdata search
#GranuleUR: S1A_IW_SLC__1SSV_20170211T141311_20170211T141339_015237_018F50_4FC8-SLC
api = GranuleQuery()
api.granule_ur('S1A_IW_SLC__1SSV_20170211T141311_20170211T141339_015237_018F50_4FC8-SLC')
print(api.hits())

1


In [9]:
granule = api.get(1)

In [10]:
granule

[{'browse_flag': False,
  'coordinate_system': 'GEODETIC',
  'data_center': 'ASF',
  'dataset_id': 'SENTINEL-1A_SLC',
  'day_night_flag': 'UNSPECIFIED',
  'granule_size': '2458.79',
  'id': 'G1370955137-ASF',
  'links': [{'href': 'https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_SLC__1SSV_20170211T141311_20170211T141339_015237_018F50_4FC8.zip',
    'hreflang': 'en-US',
    'rel': 'http://esipfed.org/ns/fedsearch/1.1/data#'},
   {'href': 'https://vertex.daac.asf.alaska.edu/',
    'hreflang': 'en-US',
    'inherited': True,
    'rel': 'http://esipfed.org/ns/fedsearch/1.1/data#'}],
  'online_access_flag': True,
  'orbit_calculated_spatial_domains': [{'orbit_number': '15237'}],
  'original_format': 'ECHO10',
  'polygons': [['47.495983 -119.080666 47.890465 -122.433319 46.217083 -122.811638 45.823486 -119.56459 47.495983 -119.080666']],
  'producer_granule_id': 'S1A_IW_SLC__1SSV_20170211T141311_20170211T141339_015237_018F50_4FC8',
  'time_end': '2017-02-11T14:13:39.000Z',
  'time_start': '2017

In [11]:
#C1214470488-ASF  #Sentinel-1A SLCS
# echo_collection_id=C1327985661-ASF #Sentinel-1B SLCS

# pyCMR

In [13]:
from pyCMR.pyCMR import CMR
cmr = CMR("/Users/scott/Documents/GitHub/pyCMR/cmr.cfg")

In [14]:
results = cmr.searchCollection(keyword='precipitation')
print(len(results))

100


In [15]:
results = cmr.searchGranule(granule_ur='S1A_IW_SLC__1SSV_20170211T141311_20170211T141339_015237_018F50_4FC8-SLC')
print(results) #slow, but works!

[{'concept-id': 'G1370955137-ASF', 'collection-concept-id': 'C1214470488-ASF', 'revision-id': '4', 'format': 'application/echo10+xml', 'Granule': {'{http://www.w3.org/2001/XMLSchema-instance}noNamespaceSchemaLocation': '', 'GranuleUR': 'S1A_IW_SLC__1SSV_20170211T141311_20170211T141339_015237_018F50_4FC8-SLC', 'InsertTime': '2017-02-17T08:46:51Z', 'LastUpdate': '2017-05-31T11:58:26Z', 'Collection': {'DataSetId': 'SENTINEL-1A_SLC'}, 'DataGranule': {'SizeMBDataGranule': '2458.79', 'ProducerGranuleId': 'S1A_IW_SLC__1SSV_20170211T141311_20170211T141339_015237_018F50_4FC8', 'DayNightFlag': 'UNSPECIFIED', 'ProductionDateTime': '2017-05-31T11:58:26Z'}, 'Temporal': {'RangeDateTime': {'BeginningDateTime': '2017-02-11T14:13:11Z', 'EndingDateTime': '2017-02-11T14:13:39Z'}}, 'Spatial': {'HorizontalSpatialDomain': {'Geometry': {'GPolygon': {'Boundary': {'Point': [{'PointLongitude': '-119.564590', 'PointLatitude': '45.823486'}, {'PointLongitude': '-122.811638', 'PointLatitude': '46.217083'}, {'PointL

In [16]:
results = cmr.searchGranule(echo_collection_id='C1327985661-ASF')
print(len(results)) #slow, but works!

100


In [17]:
results[0]

{'Granule': {'AdditionalAttributes': {'AdditionalAttribute': [{'Name': 'FLIGHT_LINE',
     'Values': {'Value': 'NULL'}},
    {'Name': 'GROUP_ID', 'Values': {'Value': 'S1B_IWDV_0448_0453_001696_095'}},
    {'Name': 'OFF_NADIR_ANGLE', 'Values': {'Value': '-1'}},
    {'Name': 'MD5SUM',
     'Values': {'Value': '6474772f12e2c057b6722cc264c0b0f5'}},
    {'Name': 'GRANULE_TYPE', 'Values': {'Value': 'SENTINEL_1B_FRAME'}},
    {'Name': 'ASCENDING_DESCENDING', 'Values': {'Value': 'DESCENDING'}},
    {'Name': 'FAR_END_LAT', 'Values': {'Value': '42.7105'}},
    {'Name': 'INSAR_STACK_SIZE', 'Values': {'Value': '1'}},
    {'Name': 'BEAM_MODE_TYPE', 'Values': {'Value': 'IW'}},
    {'Name': 'INSAR_BASELINE', 'Values': {'Value': '0'}},
    {'Name': 'CENTER_FRAME_ID', 'Values': {'Value': '2717'}},
    {'Name': 'CENTER_ESA_FRAME', 'Values': {'Value': '2717'}},
    {'Name': 'ACQUISITION_DATE', 'Values': {'Value': '2016-08-20T05:19:07Z'}},
    {'Name': 'MISSION_NAME', 'Values': {'Value': 'NULL'}},
    {'N

In [18]:
#results = cmr.searchGranule(dataset_id='SENTINEL-1B_SLC')
results = cmr.searchGranule(entry_title='SENTINEL-1B_SLC') #entry_title = dataset_id
print(len(results)) #slow, but works!
results[0]

100


{'Granule': {'AdditionalAttributes': {'AdditionalAttribute': [{'Name': 'FLIGHT_LINE',
     'Values': {'Value': 'NULL'}},
    {'Name': 'GROUP_ID', 'Values': {'Value': 'S1B_IWDV_0448_0453_001696_095'}},
    {'Name': 'OFF_NADIR_ANGLE', 'Values': {'Value': '-1'}},
    {'Name': 'MD5SUM',
     'Values': {'Value': '6474772f12e2c057b6722cc264c0b0f5'}},
    {'Name': 'GRANULE_TYPE', 'Values': {'Value': 'SENTINEL_1B_FRAME'}},
    {'Name': 'ASCENDING_DESCENDING', 'Values': {'Value': 'DESCENDING'}},
    {'Name': 'FAR_END_LAT', 'Values': {'Value': '42.7105'}},
    {'Name': 'INSAR_STACK_SIZE', 'Values': {'Value': '1'}},
    {'Name': 'BEAM_MODE_TYPE', 'Values': {'Value': 'IW'}},
    {'Name': 'INSAR_BASELINE', 'Values': {'Value': '0'}},
    {'Name': 'CENTER_FRAME_ID', 'Values': {'Value': '2717'}},
    {'Name': 'CENTER_ESA_FRAME', 'Values': {'Value': '2717'}},
    {'Name': 'ACQUISITION_DATE', 'Values': {'Value': '2016-08-20T05:19:07Z'}},
    {'Name': 'MISSION_NAME', 'Values': {'Value': 'NULL'}},
    {'N

In [19]:
# Add search bbox
results = cmr.searchGranule(entry_title='SENTINEL-1B_SLC', 
                            temporal="2017-10-10T00:00:00Z,2018-01-07T00:00:00Z",
                            bounding_box='-122.4,47.4,-122.1,47.7') 
print(len(results)) #slow, but works!
results[0]
#for res in results:
#    print(res.getDownloadUrl()) #returns none

35


{'Granule': {'AdditionalAttributes': {'AdditionalAttribute': [{'Name': 'ACQUISITION_DATE',
     'Values': {'Value': '2017-10-12T01:53:38.212757'}},
    {'Name': 'ASCENDING_DESCENDING', 'Values': {'Value': 'ASCENDING'}},
    {'Name': 'ASF_PLATFORM', 'Values': {'Value': 'Sentinel-1B'}},
    {'Name': 'BEAM_MODE', 'Values': {'Value': 'IW'}},
    {'Name': 'BEAM_MODE_DESC',
     'Values': {'Value': 'Interferometric Wide. 250 km swath, 5 m x 20 m spatial resolution and burst synchronization for interferometry. IW is considered to be the standard mode over land masses.'}},
    {'Name': 'BEAM_MODE_TYPE', 'Values': {'Value': 'IW'}},
    {'Name': 'BYTES', 'Values': {'Value': '4698416381'}},
    {'Name': 'CENTER_ESA_FRAME', 'Values': {'Value': '958'}},
    {'Name': 'CENTER_FRAME_ID', 'Values': {'Value': '155'}},
    {'Name': 'CENTER_LAT', 'Values': {'Value': '47.9307'}},
    {'Name': 'CENTER_LON', 'Values': {'Value': '-120.7291'}},
    {'Name': 'DOPPLER', 'Values': {'Value': '0'}},
    {'Name': 'F